# Hyperparameter tuning with Ray Tune

Here using the MNIST dataset and the Pytorch model, the learning rate and decay are tuned.

In [1]:
from pathlib import Path
import numpy as np

# Import pytorch modules
import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

# Import ray modules
import ray
from ray import air, tune
from ray.air import session
from ray.tune.schedulers import ASHAScheduler

### Creating Pytorch models and the standard training code.

In [2]:
class SimpleConvNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        x = F.log_softmax(x, dim=1)
        return x

In [3]:
def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total * 100

### Define trainable function

![Distributed training per trial](https://docs.ray.io/en/latest/_images/tuning_use_case.png)

In [4]:
# trainable function requires a config argument.
def train_mnist(config: dict):
    data_path = config["data_path"]
    mnist_transforms = transforms.Compose(
        [
            transforms.ToTensor(),
            transforms.Normalize((0.1307, ), (0.3081, ))
        ]
    )

    train_loader = DataLoader(
        datasets.MNIST(data_path, train=True, download=True, transform=mnist_transforms),
        batch_size=256,
        shuffle=True
    )
    test_loader = DataLoader(
        datasets.MNIST(data_path, train=False, transform=mnist_transforms),
        batch_size=256,
        shuffle=True
    )

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = SimpleConvNet()
    model.to(device)

    optimizer = torch.optim.AdamW(model.parameters(), lr=config["lr"], weight_decay=config["weight_decay"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        # This will be used by the scheduler to decide the next trial and see the result in the dashboard
        session.report({"mean_accuracy": acc})

        if i % 5 == 0:
            torch.save(model.state_dict(), "./model.pth")

In [5]:
data_path = Path.cwd().parent / "data"
data_path.mkdir(exist_ok=True, parents=True)
print(f"Data path: {data_path}")

# Define the search space for hyperparameters
search_space = {
    "lr": tune.sample_from(lambda spec: 10 ** (-10 * np.random.rand())),
    "weight_decay": tune.uniform(0.1, 0.9),
    "data_path": str(data_path), # Not a hyperparameter
}

ray.init()

# Download the dataset first
datasets.MNIST(data_path, train=True, download=True)

tuner = tune.Tuner(
    train_mnist,
    param_space=search_space,
    tune_config=tune.TuneConfig(
        num_samples=10,
        scheduler=ASHAScheduler(metric="mean_accuracy", mode="max"),
    )
)
results = tuner.fit()

(train_mnist pid=14084) Caught sync error: Sync process failed: GetFileInfo() yielded path 'C:/Users/lcglab/ray_results/train_mnist_2023-07-25_17-08-12/train_mnist_66564_00005_5_weight_decay=0.4957_2023-07-25_17-08-12/events.out.tfevents.1690272502.DESKTOP-TSBOB2F', which is outside base dir 'C:\Users\lcglab\ray_results\train_mnist_2023-07-25_17-08-12\train_mnist_66564_00005_5_weight_decay=0.4957_2023-07-25_17-08-12\'. Retrying after sleeping for 1.0 seconds...
(train_mnist pid=14084) Could not upload checkpoint to c://\Users\lcglab\ray_results\train_mnist_2023-07-25_17-08-12\train_mnist_66564_00005_5_weight_decay=0.4957_2023-07-25_17-08-12 even after 3 retries.Please check if the credentials expired and that the remote filesystem is supported. For large checkpoints or artifacts, consider increasing `SyncConfig(sync_timeout)` (current value: 1800 seconds).
2023-07-25 17:11:39,678	WARNING tune.py:1122 -- Trial Runner checkpointing failed: Sync process failed: GetFileInfo() yielded path 

The results can be viewed in the Tensorboard.
```bash
tensorboard --logdir ~/ray_results
```

In [19]:
for result in results:
    print(result.metrics)

{'mean_accuracy': 8.32, 'time_this_iter_s': 38.870152950286865, 'done': True, 'training_iteration': 1, 'trial_id': '66564_00000', 'date': '2023-07-25_17-09-01', 'timestamp': 1690272541, 'time_total_s': 38.870152950286865, 'pid': 14260, 'hostname': 'DESKTOP-TSBOB2F', 'node_ip': '127.0.0.1', 'config': {'lr': 5.109337714927793e-08, 'weight_decay': 0.32107756986403235, 'data_path': 'c:\\Users\\lcglab\\Documents\\Repo\\ray-intro\\data'}, 'time_since_restore': 38.870152950286865, 'iterations_since_restore': 1, 'experiment_tag': '0_weight_decay=0.3211'}
{'mean_accuracy': 10.100000000000001, 'time_this_iter_s': 38.728158473968506, 'done': True, 'training_iteration': 1, 'trial_id': '66564_00001', 'date': '2023-07-25_17-09-01', 'timestamp': 1690272541, 'time_total_s': 38.728158473968506, 'pid': 6732, 'hostname': 'DESKTOP-TSBOB2F', 'node_ip': '127.0.0.1', 'config': {'lr': 0.32178576700003203, 'weight_decay': 0.8501266376886318, 'data_path': 'c:\\Users\\lcglab\\Documents\\Repo\\ray-intro\\data'}, 